## Spotify

### Optional -Spotify Top 200 Artist_US Daily Chart

In [9]:
import requests
import pandas as pd
from bs4 import BeautifulSoup
from urllib.parse import urljoin

KWORB_DAILY_URL = "https://kworb.net/spotify/country/us_daily.html"
BASE = "https://kworb.net"

In [10]:
def normalize_url(href):
    """
    Normalize track or artist href from Kworb.
    Ensures:
    - no "../"
    - includes /spotify/
    - full absolute URL
    """
    if not href:
        return None

    # convert relative → absolute (removes ../)
    url = urljoin(BASE + "/", href)

    # auto fix missing /spotify/ for tracks
    if "/track/" in url and "/spotify/" not in url:
        url = url.replace("/track/", "/spotify/track/")

    # auto fix missing /spotify/ for artists
    if "/artist/" in url and "/spotify/" not in url:
        url = url.replace("/artist/", "/spotify/artist/")

    return url

In [11]:
def extract_links(td):
    links = td.find_all("a")
    artist_url = None
    track_url = None

    for a in links:
        raw = a.get("href", "")
        if not raw:
            continue

        norm = normalize_url(raw)

        if "/track/" in norm:
            track_url = norm
        elif "/artist/" in norm:
            artist_url = norm

    return artist_url, track_url

In [12]:
def get_artist_songs_url_from_track(track_url):
    try:
        r = requests.get(track_url)
        soup = BeautifulSoup(r.text, "html.parser")

        a = soup.find("a", href=lambda x: x and "/artist/" in x)
        if not a:
            return None

        href = a.get("href", "")
        norm = normalize_url(href)

        artist_id = norm.split("/")[-1].replace(".html", "")

        final_url = f"{BASE}/spotify/artist/{artist_id}_albums.html"
        return final_url

    except:
        return None

In [13]:
def get_us_daily_tracks():
    r = requests.get(KWORB_DAILY_URL)
    soup = BeautifulSoup(r.text, "html.parser")

    table = soup.find("table")
    rows = table.find_all("tr")[1:]  # skip header

    data = []

    for row in rows:
        cols = row.find_all("td")

        if len(cols) < 7:
            continue

        rank = cols[0].text.strip()

        # The "Artist and Title" column e.g. "Taylor Swift - The Fate of Ophelia"
        artist_title = cols[2].text.strip()

        # split on " - "
        if " - " in artist_title:
            artist_name, track_name = artist_title.split(" - ", 1)
        else:
            artist_name = artist_title
            track_name = ""

        # raw links
        _, track_url = extract_links(cols[2])

        # true artist_songs_url
        artist_albums_url = get_artist_songs_url_from_track(track_url)
        
        # Strer
        daily_streams = cols[6].text.strip()
        streams_plus = cols[7].text.strip() if len(cols) > 7 else None
        seven_day = cols[8].text.strip() if len(cols) > 8 else None
        seven_day_plus = cols[9].text.strip() if len(cols) > 9 else None
        total_streams = cols[10].text.strip()

        data.append({
            "rank": rank,
            "artist_name": artist_name,
            "track_name": track_name,
            "daily_streams": daily_streams,
            "streams_plus": streams_plus,
            "seven_day": seven_day,
            "seven_day_plus": seven_day_plus,
            "total_streams": total_streams,
            "artist_url": artist_albums_url,
            "track_url": track_url
        })

    return pd.DataFrame(data)

df_chart = get_us_daily_tracks()
df_chart.head(10)

,rank,artist_name,track_name,daily_streams,streams_plus,seven_day,seven_day_plus,total_streams,artist_url,track_url
0,1,Brenda Lee,Rockin' Around The Christmas Tree,"1,767,047","+158,125","8,541,112","+809,842","481,209,886",https://kworb.net/spotify/artist/4cPHsZM98sKzm...,https://kworb.net/spotify/track/2EjXfH91m7f8Hi...
1,2,Bobby Helms,Jingle Bell Rock,"1,483,931","+155,490","7,211,679","+636,964","411,036,059",https://kworb.net/spotify/artist/38EmEgXkgK51M...,https://kworb.net/spotify/track/7vQbuQcyTflfCI...
2,3,Mariah Carey,All I Want for Christmas Is You,"1,471,815","+86,257","7,621,122","+529,462","493,434,774",https://kworb.net/spotify/artist/4iHNK0tOyZPYn...,https://kworb.net/spotify/track/0bYg9bo50gSsH3...
3,4,Andy Williams,It's the Most Wonderful Time of the Year,"1,346,399","+97,929","6,197,702","+718,816","341,396,649",https://kworb.net/spotify/artist/4sj6D0zlMOl25...,https://kworb.net/spotify/track/1IcR6RlgvDczfv...
4,5,Wham!,Last Christmas,"1,292,211","+88,407","6,634,322","+509,321","357,424,665",https://kworb.net/spotify/artist/5lpH0xAS4fVfL...,https://kworb.net/spotify/track/2FRnf9qhLbvw8f...
5,6,Nat King Cole,The Christmas Song (Merry Christmas To You),"1,164,999","+92,653","5,667,034","+541,606","235,956,983",https://kworb.net/spotify/artist/7v4imS0moSyGd...,https://kworb.net/spotify/track/4PS1e8f2LvuTFg...
6,7,Dean Martin,Let It Snow! Let It Snow! Let It Snow!,"1,127,253","+104,322","5,491,127","+494,136","226,324,369",https://kworb.net/spotify/artist/49e4v89VmlDcF...,https://kworb.net/spotify/track/2uFaJJtFpPDc5P...
7,8,HUNTR/X,"Golden (w/ Ejae, AUDREY NUNA, REI AMI, KPop De...","1,073,069","-8,609","7,743,945","-183,058","261,791,548",https://kworb.net/spotify/artist/2yNNYQBChuox9...,https://kworb.net/spotify/track/1CPZ5BxNNd0n0n...
8,9,Ariana Grande,Santa Tell Me,"1,071,961","+67,222","5,558,666","+404,291","297,436,759",https://kworb.net/spotify/artist/66CXWjxzNUsdJ...,https://kworb.net/spotify/track/0lizgQ7Qw35od7...
9,10,The Ronettes,Sleigh Ride,"1,068,851","+82,657","5,234,886","+435,178","193,693,197",https://kworb.net/spotify/artist/7CyeXFnOrfC1N...,https://kworb.net/spotify/track/7fUYUcWy853HS8...


In [14]:
from datetime import datetime

today = datetime.now().strftime("%Y%m%d")

df_chart.to_csv(f"outputs_kworb/spotify_us_daily_artist_{today}.csv", index=False)

# ==== SAVE JSON ====
json_path = f"outputs_kworb/spotify_us_daily_artists_{today}.json"
df_chart.to_json(json_path, orient="records", force_ascii=False, indent=2)

print("🎉 JSON saved:", json_path)

🎉 JSON saved: outputs_kworb/spotify_us_daily_artists_20251201.json


In [ ]:
# Extract artist_id and track_id

In [16]:
def extract_artist_id(url):
    if pd.isna(url) or not isinstance(url, str):
        return None
    filename = url.split("/")[-1]  # 06HL4z0CvFAxyc27GXpf02_songs.html
    return filename.replace("_songs.html", "").replace("_albums.html", "")


In [17]:
def extract_track_id(url):
    if pd.isna(url) or not isinstance(url, str):
        return None
    filename = url.split("/")[-1]  # 1jJci4qxiYcOHhQR247rEU.html
    return filename.replace(".html", "").strip()


In [19]:
import pandas as pd
df_chart = pd.read_csv(f"outputs_kworb/spotify_us_daily_artist_20251120.csv")

df_chart["artist_id"] = df_chart["artist_url"].apply(extract_artist_id)
df_chart["track_id"]  = df_chart["track_url"].apply(extract_track_id)

df_chart.to_csv(f"outputs_kworb/spotify_us_daily_chart_with_ids_20251120.csv", index=False)


### Step 2- Spotify Top 10 Albums per each artist in chart by monthly listeners (2500 artists)

In [1]:
import os
import requests
import pandas as pd
from bs4 import BeautifulSoup
from urllib.parse import urljoin
from datetime import datetime

In [2]:
# === Step 1: Load your monthly listener artist file ===
today = datetime.now().strftime("%Y%m%d")

input_path = f"outputs_kworb/spotify_monthly_listeners_{today}.csv"
df_chart = pd.read_csv(input_path)

print("Loaded file:", input_path)
print("Total artists:", len(df_chart))

Loaded file: outputs_kworb/spotify_monthly_listeners_20251201.csv
Total artists: 2500


In [3]:
BASE = "https://kworb.net"

def normalize_url(href):
    if not href:
        return None

    url = urljoin(BASE + "/", href)

    # Fix missing /spotify/
    if "/track/" in url and "/spotify/" not in url:
        url = url.replace("/track/", "/spotify/track/")
    if "/artist/" in url and "/spotify/" not in url:
        url = url.replace("/artist/", "/spotify/artist/")

    return url

In [4]:
def extract_artist_id(artist_url):
    """
    Extract Spotify artist_id from URL:
    https://kworb.net/spotify/artist/<artist_id>_albums.html
    """
    if not artist_url:
        return None
    filename = artist_url.split("/")[-1]
    artist_id = filename.replace("_albums.html", "").strip()
    return artist_id


In [5]:
def scrape_artist_top10(artist_name, artist_id, artist_url):
    try:
        r = requests.get(artist_url, headers={"User-Agent": "Mozilla/5.0"})
        soup = BeautifulSoup(r.text, "html.parser")

        # 只会找到一张 table，这是正确的
        table = soup.find("table")
        if table is None:
            print(f"⚠️ No table for {artist_name}")
            return pd.DataFrame([])

        rows = table.find_all("tr")[1:]  # skip header

        results = []
        for tr in rows[:10]:
            tds = tr.find_all("td")

            album_title = tds[0].text.strip()
            streams = tds[1].text.strip()
            daily = tds[2].text.strip()

            results.append({
                "artist_name": artist_name,
                "artist_id": artist_id,
                "album_title": album_title,
                "streams": streams,
                "daily": daily,
                "artist_url": artist_url
            })

        return pd.DataFrame(results)

    except Exception as e:
        print(f"❗ Error scraping {artist_name}: {e}")
        return pd.DataFrame([])


In [6]:
def scrape_all_artists_top10(df_chart):
    all_rows = []

    # Ensure deduplication by artist_url
    artists = df_chart.dropna(subset=["artist_url"]).drop_duplicates("artist_url")

    print("Total unique artist URLs:", len(artists))

    for _, row in artists.iterrows():
        artist_name = row["artist_name"]
        artist_url = normalize_url(row["artist_url"])
        artist_id = extract_artist_id(artist_url)

        print(f"🎵 Scraping Top 10 for {artist_name} ({artist_id})...")

        df_artist = scrape_artist_top10(artist_name, artist_id, artist_url)
        all_rows.append(df_artist)

    return pd.concat(all_rows, ignore_index=True)


In [7]:
df_top10 = scrape_all_artists_top10(df_chart)

output_path = f"outputs_kworb/spotify_artist_top10_ablums_{today}.csv"
df_top10.to_csv(output_path, index=False)

print("🎉 DONE! File saved:", output_path)
print("Total rows:", len(df_top10))

Total unique artist URLs: 2500
🎵 Scraping Top 10 for The Weeknd (1Xyo4u8uXC1ZmMpatF05PJ)...
🎵 Scraping Top 10 for Bruno Mars (0du5cEVh5yTK9QJze8zA0C)...
🎵 Scraping Top 10 for Taylor Swift (06HL4z0CvFAxyc27GXpf02)...
🎵 Scraping Top 10 for Justin Bieber (1uNFoZAHBGtllmzznpCI3s)...
🎵 Scraping Top 10 for Ariana Grande (66CXWjxzNUsdJxJ2JdwvnR)...
🎵 Scraping Top 10 for Rihanna (5pKCCKE2ajJHZ9KAiaK11H)...
🎵 Scraping Top 10 for Lady Gaga (1HY2Jd0NmPuamShAr6KMms)...
🎵 Scraping Top 10 for Ed Sheeran (6eUKZXaKkcviH0Ku9w2n3V)...
🎵 Scraping Top 10 for Billie Eilish (6qqNVTkY8uBg9cP3Jd7DAH)...
🎵 Scraping Top 10 for Coldplay (4gzpq5DPGxSnKTe4SA8HAU)...
🎵 Scraping Top 10 for Drake (3TVXtAsR1Inumwj472S9r4)...
🎵 Scraping Top 10 for Bad Bunny (4q3ewBCX7sLwd24euuV69X)...
🎵 Scraping Top 10 for David Guetta (1Cs0zKBU1kc0i8ypK3B9ai)...
🎵 Scraping Top 10 for Sia (5WUlDfRSoLAfcVSX1WnrxN)...
🎵 Scraping Top 10 for Sabrina Carpenter (74KM79TiuVKeVCqs8QtB0B)...
🎵 Scraping Top 10 for Maroon 5 (04gDigrS5kc9YWfZHwBET

In [8]:
# ==== SAVE JSON ====
json_path = f"outputs_kworb/spotify_artist_top10_albums_{today}.json"
df_top10.to_json(json_path, orient="records", force_ascii=False, indent=2)

print("🎉 JSON saved:", json_path)

🎉 JSON saved: outputs_kworb/spotify_artist_top10_albums_20251201.json


### Step 1- Spotify top artists by monthly listeners

In [41]:
import requests
import pandas as pd
from bs4 import BeautifulSoup
from urllib.parse import urljoin
from datetime import datetime

URL = "https://kworb.net/spotify/listeners.html"
HEADERS = {"User-Agent": "Mozilla/5.0"}


def build_artist_albums_url(href):
    if not href:
        return None
    BASE = "https://kworb.net"
    full_url = urljoin(BASE, href)   # safer than manual string + '/'
    full_url = full_url.replace("_songs.html", "_albums.html")
    return full_url


def scrape_monthly_listeners():
    r = requests.get(URL, headers=HEADERS)
    soup = BeautifulSoup(r.text, "html.parser")

    table = soup.find("table")
    if table is None:
        raise ValueError("❗ No table found on listeners page!")

    rows = table.find_all("tr")[1:]

    data = []
    for tr in rows:
        tds = tr.find_all("td")
        if len(tds) < 6:
            continue

        rank = tds[0].text.strip()

        raw_href = tds[1].find("a")["href"] if tds[1].find("a") else None
        artist_name = tds[1].text.strip()
        artist_url = build_artist_albums_url(raw_href)

        listeners = tds[2].text.strip()
        daily_change = tds[3].text.strip()
        peak_rank = tds[4].text.strip()
        peak_listeners = tds[5].text.strip()

        data.append({
            "rank": rank,
            "artist_name": artist_name,
            "artist_url": artist_url,
            "listeners": listeners,
            "daily_change": daily_change,
            "peak_rank": peak_rank,
            "peak_listeners": peak_listeners
        })

    return pd.DataFrame(data)

In [42]:
# ==== RUN & SAVE ====
df = scrape_monthly_listeners()

today = datetime.now().strftime("%Y%m%d")
csv_path = f"outputs_kworb/spotify_monthly_listeners_{today}.csv"
json_path = f"outputs_kworb/spotify_monthly_listeners_{today}.json"

df.to_csv(csv_path, index=False)
df.to_json(json_path, orient="records", indent=2)

print("🎉 DONE! Files saved:")
print("📄 CSV :", csv_path)
print("📄 JSON:", json_path)
print("Total rows:", len(df))

🎉 DONE! Files saved:
📄 CSV : outputs_kworb/spotify_monthly_listeners_20251201.csv
📄 JSON: outputs_kworb/spotify_monthly_listeners_20251201.json
Total rows: 2500


## Youtube

### Trending Music Video

In [16]:
import requests
import pandas as pd
from bs4 import BeautifulSoup
from urllib.parse import urlparse, parse_qs

BASE_URL = "https://kworb.net/youtube/trending.html"

In [17]:
def extract_youtube_video_id(url):
    """extract video_id."""
    if not isinstance(url, str):
        return None
    if "youtube.com/watch" in url:
        parsed = urlparse(url)
        return parse_qs(parsed.query).get("v", [None])[0]
    return None

In [18]:
def scrape_kworb_youtube_trending():
    headers = {"User-Agent": "Mozilla/5.0"}
    r = requests.get(BASE_URL, headers=headers)
    soup = BeautifulSoup(r.text, "html.parser")

    table = soup.find("table")
    rows = table.find_all("tr")[1:]  # skip header

    data = []

    for tr in rows:
        tds = tr.find_all("td")
        if len(tds) < 5:
            continue

        rank = tds[0].text.strip()
        rank_change = tds[1].text.strip()

        # Artist - Song Name (video title)
        full_title = tds[2].text.strip()
        if " - " in full_title:
            artist_name, video_name = full_title.split(" - ", 1)
        else:
            artist_name = full_title
            video_name = full_title

        # YouTube link
        raw_href = a_tag["href"] if a_tag else None

        if raw_href:
            if raw_href.startswith("http"):
                youtube_url = raw_href                  # full youtube URL
            else:
                youtube_url = urljoin(BASE, raw_href)  # kworb relative → absolute
        else:
            youtube_url = None

        # "#" column (view count)
        view_count = tds[3].text.strip()

        # Highlights text (usually contains emoji + short summary)
        highlights = tds[4].text.strip()

        data.append({
            "rank": rank,
            "rank_change": rank_change,
            "artist_name": artist_name,
            "video_name": video_name,
            "view_count": view_count,
            "highlights": highlights,
            "youtube_video_url": youtube_url
        })

    return pd.DataFrame(data)

In [19]:
# ===== RUN & EXPORT =====
def export_youtube_trending():
    df = scrape_kworb_youtube_trending()

    # date stamp
    today = datetime.now().strftime("%Y%m%d")

    # output folder
    output_dir = "outputs_kworb"
    os.makedirs(output_dir, exist_ok=True)

    # output route
    csv_path = f"{output_dir}/youtube_trending_worldwide_{today}.csv"
    json_path = f"{output_dir}/youtube_trending_worldwide_{today}.json"

    # export
    df.to_csv(csv_path, index=False, encoding="utf-8-sig")
    df.to_json(json_path, orient="records", indent=2)

    print(f"🎉 DONE! Saved CSV → {csv_path}")
    print(f"🎉 DONE! Saved JSON → {json_path}")
    print("Total rows:", len(df))


# ===== EXECUTE =====
export_youtube_trending()

🎉 DONE! Saved CSV → outputs_kworb/youtube_trending_worldwide_20251130.csv
🎉 DONE! Saved JSON → outputs_kworb/youtube_trending_worldwide_20251130.json
Total rows: 191
